In [36]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf

In [37]:
# The probability margin is the absolute value of the difference
# between the two teams in terms of who is more likely to win: np.abs(df['prob1'] - df['prob2']).
# There is also probtie which predicts the probability the game ends up level.
# Simple three-layer neural network to anticipate which
# bucket of probability margin a given match falls into:
    # If X is less than the mean probability margin (MPM), it's "WITHIN MEAN PROBABILITY MARGIN."
    # If X is greater than or equal to MPM, but less than MPM + 1 standard deviation (SDPM), it's "SLIGHT PROBABILITY ADVANTAGE."
    # If X is greater than MPM + SDPM, but less than MPM + (SDPM * 1.5), it's "MODERATE PROBABILITY ADVANTAGE."
    # If X is greater than MPM + (SDPM * 1.5), meaning it is an "outlier", it's "SIGNIFICANT PROBABILITY ADVANTAGE."
# We will use statistics from the Barclays Premier League from the start of the 2018-19 season.

df = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv")
df = df[(df.league=="Barclays Premier League") & (df.season >= 2018)]

In [38]:
# A three-layer model is idea because it has enough filtering capability to generalize and get rid of the noise.
# That said, we will throw in an l2 regularizer just to trim it even further.
def load_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(8, kernel_regularizer=tf.keras.regularizers.l2(0.001), activation='relu'))
    model.add(tf.keras.layers.Dense(8, activation='relu'))
    model.add(tf.keras.layers.Dense(4, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [39]:
# The y variables need to be converted from their existing state after train_test_split to categorical.
# That said, it's a bit more than just calling to_categorical...the array in question has to be factorized, then converted
# to a NumPy array, then converted to categorical.
# This results in a four-column array of one-hot values for the variables.
# We have to do it twice, so best to create a basic function for it.
def cat(array):
    array = tf.keras.utils.to_categorical(np.asarray(pd.factorize(array)[0]))
    return array

In [40]:
# OK, now the fun part. Probability margin, MPM, and SDPM generation.
df['prob_margin'] = np.abs(df['prob1'] - df['prob2'])
mpm = np.mean(df['prob_margin'])
sdpm = np.std(df['prob_margin'])

In [41]:
# Empty list.
pm = []

In [42]:
# It's bucketing time.
for x in df['prob_margin']:
    if x < mpm:
        pm.append("WITHIN MEAN PROBABILITY MARGIN")
    elif (x >= mpm) & (x < (mpm + sdpm)):
        pm.append("SLIGHT PROBABILITY ADVANTAGE")
    elif (x >= (mpm + sdpm)) & (x < (mpm + (sdpm * 1.5))):
        pm.append("MODERATE PROBABILITY ADVANTAGE")
    else:
        pm.append("SIGNIFICANT PROBABILITY ADVANTAGE")

In [43]:
# Assigning PM to our new column, prob_bucket.
df['prob_bucket'] = pm

In [44]:
# Since there are four output classes, we need to have the identical number of columns as our predictor variables (4).
# We haven't used probtie yet, but it would be good to include.
X_final = df[['prob1', 'prob2', 'prob_margin', 'probtie']]
y_final = df[['prob_bucket']].copy()
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=6)

In [45]:
# Converting y_train and y_test to categorical.
y_train = cat(y_train["prob_bucket"])
y_test = cat(y_test["prob_bucket"])

In [46]:
# We want to see if there is a more optimal number of epochs.
# We will set it to tell us with a patience setting of 5.
early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=5, restore_best_weights=True)

In [47]:
# 6 epochs.
model = load_model()
history = model.fit(X_train, y_train, batch_size=128, epochs=25, validation_data=(X_test, y_test), callbacks=[early_stop])

Epoch 1/25
8/8 [==============================] - 0s 16ms/step - loss: 1.3660 - accuracy: 0.6031 - val_loss: 1.3840 - val_accuracy: 0.2149
Epoch 2/25
8/8 [==============================] - 0s 3ms/step - loss: 1.3491 - accuracy: 0.6140 - val_loss: 1.3843 - val_accuracy: 0.2456
Epoch 3/25
8/8 [==============================] - 0s 3ms/step - loss: 1.3326 - accuracy: 0.5888 - val_loss: 1.3852 - val_accuracy: 0.2368
Epoch 4/25
8/8 [==============================] - 0s 2ms/step - loss: 1.3166 - accuracy: 0.5888 - val_loss: 1.3868 - val_accuracy: 0.2281
Epoch 5/25
8/8 [==============================] - 0s 2ms/step - loss: 1.3002 - accuracy: 0.5888 - val_loss: 1.3896 - val_accuracy: 0.2281
Epoch 6/25
8/8 [==============================] - 0s 2ms/step - loss: 1.2833 - accuracy: 0.5888 - val_loss: 1.3927 - val_accuracy: 0.2281


In [48]:
# We could see the number of epochs, but easier to just tie it to an object.
num_epochs = len(history.__dict__['epoch'])

In [49]:
# Given that early_stop stopped after 6 epochs, we will run 6 as well.
# Again, since it's tied to an object, not much extra work involved.
model.fit(X_train, y_train, epochs=num_epochs, batch_size=128, verbose=1)

Epoch 1/6
8/8 [==============================] - 0s 572us/step - loss: 1.3495 - accuracy: 0.6129
Epoch 2/6
8/8 [==============================] - 0s 572us/step - loss: 1.3329 - accuracy: 0.5899
Epoch 3/6
8/8 [==============================] - 0s 572us/step - loss: 1.3169 - accuracy: 0.5888
Epoch 4/6
8/8 [==============================] - 0s 572us/step - loss: 1.3006 - accuracy: 0.5888
Epoch 5/6
8/8 [==============================] - 0s 572us/step - loss: 1.2843 - accuracy: 0.5888
Epoch 6/6
8/8 [==============================] - 0s 429us/step - loss: 1.2677 - accuracy: 0.5888


In [50]:
# Generating the predictions.
predictions = model.predict(X_test)

In [51]:
# We'll take it.
loss, accuracy = model.evaluate(y_test, predictions)

8/8 [==============================] - 0s 716us/step - loss: 1.3886 - accuracy: 1.0000


In [52]:
# For good measure to close it out.
print("Test Accuracy: {}%".format(accuracy * 100))

Test Accuracy: 100.0%
